Machine config:

**CUDA 12.4, GPU: AD104 12GB VRAM, CPU: 12/24 4.4**

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

### Baseline FP16

In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

In [ ]:
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
def splitter(n, s):
    pieces = s.split()
    return (" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n))

In [ ]:
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print('\n\n')
for n in splitter(15, result["text"]):
    print(n)

----

### ONNX-optimum

ONNX conversion (FP16)

In [ ]:
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq
from transformers import AutoProcessor

model_id = "openai/whisper-large-v3-turbo"
processor = AutoProcessor.from_pretrained(model_id)

# Export encoder and decoder to ONNX
model = ORTModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    export=True,
    provider="CUDAExecutionProvider" if torch.cuda.is_available() else "CPUExecutionProvider"
)
model.save_pretrained("whisper_lv3t_onnx_v3")
processor.save_pretrained("whisper_lv3t_onnx_v3")

Optimization

In [ ]:
import onnx
import onnxoptimizer

onnx_model = onnx.load("onnxf/whisper_lv3t_onnx_v3/encoder_model.onnx")

passes = ["fuse_bn_into_conv", "eliminate_unused_initializer"]
optimized_model = onnxoptimizer.optimize(onnx_model, passes)

onnx.save(optimized_model, "onnxf/whisper_lv3t_onnx_v3/encoder_optimized.onnx")

Quantization (Int8)

In [ ]:
# -- will be here soon --

---

### ONNX-optimum v2

In [ ]:
from pathlib import Path
from optimum.onnxruntime import (
    AutoQuantizationConfig,
    ORTModelForSpeechSeq2Seq,
    ORTQuantizer
)
from onnxruntime.quantization import QuantType

In [ ]:
model_id = "openai/whisper-large-v3-turbo"
processor = AutoProcessor.from_pretrained(model_id)

ONNX conversion (FP16)

In [ ]:
model = ORTModelForSpeechSeq2Seq.from_pretrained(
    model_id, 
    export=True,
    provider="CUDAExecutionProvider",
)
model_dir = model.model_save_dir

onnx_models = list(Path(model_dir).glob("*.onnx"))
# print(onnx_models)
quantizers = [ORTQuantizer.from_pretrained(model_dir, file_name=onnx_model) for onnx_model in onnx_models]

AutoQuantizationConfig, avc512_vnni + custom QuantType to QUInt8

In [ ]:
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
qconfig.weights_dtype = QuantType.QUInt8

for quantizer in quantizers:
    quantizer.quantize(save_dir="onnxf/whisper_lv3t_onnx_v1", quantization_config=qconfig)

----